In [1]:
%run StdPackages.ipynb

No clean-up of work-folder


In [2]:
db = Database.GpyDB(db=os.path.join(d['main'],'gams\\AbatementExample\\V01_DB.gdx'))

*Load classes:*

In [3]:
os.chdir(d['py'])
from GmsPy import Group
os.chdir(d['curr'])

# Model components

### Grouping of variables

The class ```GmsPy.Group``` specifies how to define groupings of variables. We can define groups of variables using four attributes:
* ```self.v```: List of tuples. Each tuple is length 2, with first element indicating the variable, the second indicating the condition on that variable.
* ```self.g```: Ordered set. Each element is a string referencing other groups to be included.
* ```self.neg_v```: List of tuples akin to ```self.v```. Each element is subtracted from the group.
* ```sef.neg_g```: Ordered set. Each element is a string referencing another group that is subtracted.

When specifying variables, the conditions should conform to the form used in ```DBWheels_rc``` and ```GmsPyWheels_write```.

*NB: As groups can depend on other groups, it can be important to process groups in the right order.*

In [4]:
groups = {}

#### Add a group specified as a list of tuples

Specify the list of tuples and initialize a group:

In [5]:
g1 = Group('g1', v = [('theta',db['V01_T']), 
                      ('mu', ('and', [db['V01_inp2T'], ('not', db['V01_dur'])])),
                      ('sigma', db['V01_T2ESNorm'])])

Compile:

In [6]:
groups[g1.name] = g1.compile(groups)
g1.out

{'theta': [<_Database.gpy at 0x23b2ddc3c70>],
 'mu': [('and',
    ('not', <_Database.gpy at 0x23b2ddc38b0>)])],
 'sigma': [<_Database.gpy at 0x23b2ddc86a0>]}

#### Add a group specified by referencing a group and variables

Specify list of variables + reference group 'g1'. This adds a variable 'mu' and the conditional of the mapping 'V01_NT_inp' that has been aliased with 'n','nn'. A preview of what will eventually be written:

In [7]:
c = {'s': db['V01_NT_inp'], 'alias': {'n':'nn'}}
write_gpy(**c)

'V01_NT_inp[s,nn]'

In [8]:
g2 = Group('g2', v = [('mu',c)], g=['g1'])

Compile:

In [9]:
groups[g2.name] = g2.compile(groups)
groups[g2.name].out

{'mu': [{'s': <_Database.gpy at 0x23b2dd7e940>, 'alias': {'n': 'nn'}},
  ('and',
    ('not', <_Database.gpy at 0x23b2ddc38b0>)])],
 'theta': [<_Database.gpy at 0x23b2ddc3c70>],
 'sigma': [<_Database.gpy at 0x23b2ddc86a0>]}

#### Subtract a group

Define a group as the group 'g2', but extract 'g1':

In [10]:
g3 = Group('g3', g = ['g2'], neg_g = ['g1'])

Compile:

In [11]:
groups[g3.name] = g3.compile(groups)
groups[g3.name].out

{'mu': [{'s': <_Database.gpy at 0x23b2dd7e940>, 'alias': {'n': 'nn'}}]}

In [12]:
groups[g3.name].out_neg

{}

#### Subtract a group and a variable:

In [13]:
g4 = Group('g4', g = ['g2'], neg_g = ['g1'], neg_v = [('mu', db['V01_NT_inp'])])

Compile:

In [14]:
groups[g4.name] = g4.compile(groups)
groups[g4.name].out, g4.out_neg

({'mu': [{'s': <_Database.gpy at 0x23b2dd7e940>, 'alias': {'n': 'nn'}}]},
 {'mu': [<_Database.gpy at 0x23b2dd7e940>]})

#### Collect conditions from a group:

The ```self.compile``` creates a dictionary of conditions to apply (```self.out```) and conditions to negate and apply (```self.out_neg```). The two are combined into one dictionary of nested conditions using the ```self.conditions``` property. These can be processed using e.g. the ```write_gpy``` method:

In [15]:
for g in groups.values():
    print(f"Text for group {g.name}:")
    for k,v in g.conditions.items():
        print(f"\t{write_gpy(db[k],c=v)}")

Text for group g1:
	theta[s,n]$(V01_T[s,n])
	mu[s,n,nn]$((V01_inp2T[s,n,nn] and ( not (V01_dur[s,n]))))
	sigma[s,n]$(V01_T2ESNorm[s,n,nn])
Text for group g2:
	mu[s,n,nn]$((V01_NT_inp[s,nn] or (V01_inp2T[s,n,nn] and ( not (V01_dur[s,n])))))
	theta[s,n]$(V01_T[s,n])
	sigma[s,n]$(V01_T2ESNorm[s,n,nn])
Text for group g3:
	mu[s,n,nn]$(V01_NT_inp[s,nn])
Text for group g4:
	mu[s,n,nn]$((V01_NT_inp[s,nn] and ( not (V01_NT_inp[s,n]))))
